In [2]:
import pandas as pd
import folium

In [3]:
feu_galveston = pd.read_csv("../data/FUE_Galveston.csv")
feu_galveston.head()
# you'll see that we're given latitude and longitude coordinates for each location in Galveston.

,Name,Weekly_Dropoff_Totes,Daily_Pickup_Totes,Address,Latitude,Longitude
0,Moody Gardens,0.0,0.0,"1 Hope Blvd, Galveston",29.2736,-94.8523
1,Sugar Bean,1.0,1.0,"11 Evia Main, Galveston",29.2610,-94.8704
2,MAC-IES,1.0,1.0,"1110 23rd Street, Galveston",29.2989,-94.7914
3,Good Dough,1.0,1.0,"1508 39th Street, Galveston",29.2906,-94.8085
4,Porch Cafe,1.0,1.0,"1625 East Beach Drive #2A, Galveston",29.3196,-94.7502


In [4]:
# How-to map lat and long points using geopandas: https://geopandas.org/en/latest/gallery/create_geopandas_from_pandas.html
# How-to map lat and long points using folium: https://georgetsilva.github.io/posts/mapping-points-with-folium/

# Map each point using folium
# make a list of every feu location point 
locations_df = feu_galveston[['Latitude', 'Longitude']]
locations_list = locations_df.values.tolist()
# I obtained latitude and longitude coordinates of Galvenston center at https://latitude.to/map/us/united-states/cities/galveston
galveston_map = folium.Map(location = [29.30135, -94.7977], tiles ='OpenStreetMap', zoom_start=11)
for point in range(0, len(locations_list)):
    folium.Marker(locations_list[point], popup=feu_galveston['Name'][point]).add_to(galveston_map)

# to add a route line between points in a list, use the following code 
# folium.PolyLine(list_of_lat_long_points_on_route).add_to(galveston_map) 
# source: https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/

galveston_map
